# Introduction to LLM-Based Sentiment Analysis
Here, I’m shifting gears to explore sentiment extraction using Large Language Models (LLMs). I’ll leverage OpenAI's models, and experiment with prompting of these models. This builds on the cleaned data from earlier, aiming to capture nuanced sentiment that BoW might miss. The goal remains the same: to evaluate how well these scores predict stock market reactions compared to other methods.

In [150]:
### Starting counter for run time
import time
start_time = time.time()

In [151]:
### Getting the Earning Call data
import sqlite3 as sql
import pandas as pd
import numpy as np
import openai

### Getting the needed key
%run 9-api_keys.ipynb

# Create a connection to the SQLite database
conn = sql.connect('data.db')
ECs = pd.read_sql_query("SELECT * from ECs3", conn)
conn.close()

In [152]:
# # ### Getting a sample of the data
# # ECs = ECs.sample(10, random_state=42)

# ### Getting just row 365, 1385 and 2634
# row_numbers = [365, 1385, 2634]
# ECs = ECs.iloc[row_numbers]

# ### Resetting the index
# ECs = ECs.reset_index(drop=True)

# #Looking at the shape of the data
# print(ECs.shape)

# OpenAI model

In [153]:
### Choice of model
import requests
headers = {
    "Authorization": f"Bearer {openai_key}"
}

response = requests.get("https://api.openai.com/v1/models", headers=headers)

if response.status_code == 200:
    models = response.json()["data"]
    for model in sorted(models, key=lambda m: m["id"]):
        print(model["id"])
else:
    print(f"Error {response.status_code}: {response.text}")

babbage-002
chatgpt-4o-latest
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-2024-11-20
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview-2024-12-17
gpt-4o-search-preview
gpt-4o-search-preview-2025-03-11
gpt-4o-transcribe
o1-mini
o1-mini-2024-09-12
o1-preview
o1-preview-2024-09-12
omni-moderation-2024-0

In [154]:
### Configure OpenAI client (new style)
import tiktoken
def count_tokens(text, model="gpt-3.5-turbo"):
    """
    Count the number of tokens in a given text using the specified OpenAI model.
    """
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

client = openai.OpenAI(api_key=openai_key)
def openai_llm(prompt, verbose=False, apply_template=True, temperature=0.7, max_tokens=150, model="gpt-3.5-turbo", top_p=0.95):
    """
    Send a prompt to OpenAI API (Chat or Completion) depending on the model.
    """
    system_msg = "You are a helpful analyst who evaluates sentiment in earnings call presentations and Q&As."
    user_msg = prompt.strip()

    # Handle Chat Models
    if model.startswith("gpt-"):
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ] if apply_template else [{"role": "user", "content": user_msg}]

        if verbose:
            print("=== Chat Prompt Messages ===")
            for msg in messages:
                print(f"{msg['role'].upper()}: {msg['content']}")
            print("============================")
        
        if count_tokens(prompt + '' + system_msg) > 16000:
            return np.nan
        
        else:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p
            )
            return response.choices[0].message.content.strip()

    # Handle Completion Models (e.g., text-davinci-003)
    else:
        full_prompt = f"{system_msg}\n\n{user_msg}" if apply_template else user_msg

        if verbose:
            print("=== Completion Prompt ===")
            print(full_prompt)
            print("=========================")

        response = client.completions.create(
            model=model,
            prompt=full_prompt,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p
        )
        return response.choices[0].text.strip()

In [155]:
def analyze_earnings_sentiment(presentation_text, a_text, q_text, model="gpt-3.5-turbo", temperature=0.0, max_tokens=150, what_to_analyze="P"):
    if what_to_analyze == "P":
        # Analyze only presentations
        prompt = f"""
                Analyze the sentiment of the following earnings call presentation. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {presentation_text}
                 """
        
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response
    elif what_to_analyze == "A":
        # Analyze only answers
        prompt = f"""
                Analyze the sentiment of the following earnings call answers during the Q&A. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {a_text}
                 """
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response
    elif what_to_analyze == "Q":
        # Analyze only answers
        prompt = f"""
                Analyze the sentiment of the following earnings call questions during the Q&A. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {q_text}
                 """
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response

In [156]:
def analyze_earnings_sentiment_with_EPS(presentation_text, a_text, q_text, earning_suprise, model="gpt-3.5-turbo", temperature=0.0, max_tokens=150, what_to_analyze="P"):
    if what_to_analyze == "P":
        # Analyze only presentations
        prompt = f"""
                Analyze the sentiment of the following earnings call presentation. We already know that the scalled earnings suprise (Actual EPS - Expected EPS) is {earning_suprise}. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {presentation_text}
                 """
        
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response
    elif what_to_analyze == "A":
        # Analyze only answers
        prompt = f"""
                Analyze the sentiment of the following earnings call answers during the Q&A. We already know that the scalled earnings suprise (Actual EPS - Expected EPS) is {earning_suprise}. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {a_text}
                 """
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response
    elif what_to_analyze == "Q":
        # Analyze only answers
        prompt = f"""
                Analyze the sentiment of the following earnings call questions during the Q&A. We already know that the scalled earnings suprise (Actual EPS - Expected EPS) is {earning_suprise}. Rate the sentiment on a scale from 1 (very negative) to 10 (very positive). Return only a single number as your response, with no additional text or explanation. Think step by step and consider the overall tone, language, and context of the text.

                Text:
                {q_text}
                 """
        response = openai_llm(prompt, model=model, temperature=temperature, max_tokens=max_tokens)
        return response

In [157]:
import re

def anonymize(text):
    # Step 1: Replace company names
    text = re.sub(r"([A-Z][A-Za-z0-9]*(?:\s+[A-Z][A-Za-z0-9]*)*)\s+(Earnings|Conference Call|Inc\.|Corp\.|Corporation)", r"[Company] \2", text)
    text = text.replace("[Company] Earnings", "[Company] [Quarter] Earnings")

    # Step 2: Replace executive names dynamically
    exec_pattern = r"(?:Mr\.|Ms\.|Mrs\.|Dr\.|\bturn\s+the\s+conference\s+over\s+to\b|\bon\s+the\s+call\s+today\s+are\b)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)(?:,|\s+(Executive\s+Chairman|Chief\s+Executive\s+Officer|Chief\s+Financial\s+Officer|Vice\s+President|President|CEO|CFO|COO))"
    exec_counter = 1
    exec_replacements = {}
    for match in re.finditer(exec_pattern, text, re.IGNORECASE):
        full_name = match.group(1)
        if full_name not in exec_replacements:
            exec_replacements[full_name] = f"[Executive {exec_counter}]"
            exec_counter += 1
    for name, placeholder in exec_replacements.items():
        text = text.replace(name, placeholder)

    # Step 3: Replace brands, programs, films (avoiding prior replacements)
    # Simplified pattern: Capitalized phrases followed by context words
    brand_pattern = r"\b([A-Z][A-Za-z0-9]*(?:\s+[A-Z][A-Za-z0-9]*)*)(?:\s+(Network|Series|Film|Channel|Studio|Live))"
    brand_counter = 1
    program_counter = 1
    film_counter = 1
    brand_replacements = {}
    for match in re.finditer(brand_pattern, text):
        entity = match.group(1)
        context = match.group(2).lower()
        # Skip if already replaced as Company or Executive
        if not re.search(r"\[Company\]|\[Executive \d+\]", entity) and entity not in brand_replacements:
            if "series" in context or "live" in context:
                brand_replacements[entity] = f"[Program {program_counter}]"
                program_counter += 1
            elif "film" in context or "studio" in context:
                brand_replacements[entity] = f"[Film {film_counter}]"
                film_counter += 1
            else:
                brand_replacements[entity] = f"[Brand {brand_counter}]"
                brand_counter += 1
    for entity, placeholder in brand_replacements.items():
        text = text.replace(entity, placeholder)

    # Step 4: Replace platforms
    platform_pattern = r"(?:on|with|to|via)\s+([A-Z][A-Za-z0-9]*(?:\s+[A-Z][A-Za-z0-9]*)*)(?:\s+(platform|service|streaming|TV|mobile))"
    platform_counter = 1
    platform_replacements = {}
    for match in re.finditer(platform_pattern, text, re.IGNORECASE):
        platform = match.group(1)
        if platform not in platform_replacements and not re.search(r"\[Company\]|\[Executive \d+\]|\[Brand \d+\]|\[Program \d+\]|\[Film \d+\]", platform):
            platform_replacements[platform] = f"[Platform {platform_counter}]"
            platform_counter += 1
    for platform, placeholder in platform_replacements.items():
        text = text.replace(platform, placeholder)

    # Step 5: Replace dates and quarters
    text = re.sub(r"(First|Second|Third|Fourth)\s+Quarter\s+\d{4}", "[Quarter]", text, flags=re.IGNORECASE)
    text = re.sub(r"\b(January|February|March|April|May|June|July|August|September|October|November|December)\b", "[Month]", text, flags=re.IGNORECASE)

    # Clean up extra whitespace
    text = " ".join(text.split())
    return text

# Running the models

**Unfortunately, the API key expired while running the code below, so this section has been shortened.**

In [158]:
# Define a function to process each row
def process_row(row, what_to_analyze="P"):
    pres = anonymize(row['P'])
    ans = anonymize(row['A'])
    quest = anonymize(row['Q'])
    surpdec = row['SurpDec']
    
    openai_score = analyze_earnings_sentiment(
        presentation_text=pres, 
        a_text=ans, 
        q_text=quest,
        model="gpt-3.5-turbo", 
        temperature=0, 
        max_tokens=150,
        what_to_analyze=what_to_analyze
    )
    
    # openai_eps_score = analyze_earnings_sentiment_with_EPS(
    #     presentation_text=pres, 
    #     earning_suprise=surpdec, 
    #     a_text=ans, 
    #     q_text=quest,
    #     model="gpt-3.5-turbo", 
    #     temperature=0, 
    #     max_tokens=150,
    #     what_to_analyze=what_to_analyze
    # )
    
    return pd.Series([openai_score])

# Apply the function to the dataset
ECs[['openai_P']] = ECs.apply(process_row, axis=1)

In [159]:
# # Define a function to process each row
# def process_row(row, what_to_analyze="A"):
#     pres = anonymize(row['P'])
#     ans = anonymize(row['A'])
#     quest = anonymize(row['Q'])
#     surpdec = row['SurpDec']
    
#     openai_score = analyze_earnings_sentiment(
#         presentation_text=pres, 
#         a_text=ans, 
#         q_text=quest,
#         model="gpt-3.5-turbo", 
#         temperature=0, 
#         max_tokens=150,
#         what_to_analyze=what_to_analyze
#     )
    
#     openai_eps_score = analyze_earnings_sentiment_with_EPS(
#         presentation_text=pres, 
#         earning_suprise=surpdec, 
#         a_text=ans, 
#         q_text=quest,
#         model="gpt-3.5-turbo", 
#         temperature=0, 
#         max_tokens=150,
#         what_to_analyze=what_to_analyze
#     )
    
#     return pd.Series([openai_score, openai_eps_score])

# # Apply the function to the dataset
# ECs[['openai_A', 'openai_eps_A']] = ECs.apply(process_row, axis=1)

In [160]:
# # Define a function to process each row
# def process_row(row, what_to_analyze="Q"):
#     pres = anonymize(row['P'])
#     ans = anonymize(row['A'])
#     quest = anonymize(row['Q'])
#     surpdec = row['SurpDec']
    
#     openai_score = analyze_earnings_sentiment(
#         presentation_text=pres, 
#         a_text=ans, 
#         q_text=quest,
#         model="gpt-3.5-turbo", 
#         temperature=0, 
#         max_tokens=150,
#         what_to_analyze=what_to_analyze
#     )
    
#     openai_eps_score = analyze_earnings_sentiment_with_EPS(
#         presentation_text=pres, 
#         earning_suprise=surpdec, 
#         a_text=ans, 
#         q_text=quest,
#         model="gpt-3.5-turbo", 
#         temperature=0, 
#         max_tokens=150,
#         what_to_analyze=what_to_analyze
#     )
    
#     return pd.Series([openai_score, openai_eps_score])

# # Apply the function to the dataset
# ECs[['openai_Q', 'openai_eps_Q']] = ECs.apply(process_row, axis=1)

In [161]:
### Saving just the openai_P data
openai = ECs['openai_P']

### Saving the data
conn = sql.connect('data.db')
ECs.to_sql('ECs4', conn, if_exists='replace', index=False)
openai.to_sql('openai', conn, if_exists='replace', index=False)
conn.close()

In [162]:
### Stopping the timer
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")

Execution time: 3565.87 seconds
